## Thuật toán sử dụng phương pháp hàm ngược để giải bài toán nội suy 

In [7]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_and_validate_inverse_data(file_x, file_y):
    """
    Tải dữ liệu từ file và kiểm tra các điều kiện cho nội suy ngược.
    """
    try:
        x_coords = np.loadtxt(file_x, ndmin=1).astype(float)
        y_coords = np.loadtxt(file_y, ndmin=1).astype(float)
    except (FileNotFoundError, ValueError):
        raise IOError("LỖI: Không tìm thấy file hoặc file chứa dữ liệu không hợp lệ.")
    
    # Kiểm tra số lượng điểm
    if len(x_coords) != len(y_coords):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
        
    # KIỂM TRA QUAN TRỌNG: Các giá trị y không được trùng lặp
    # vì chúng sẽ là các mốc nội suy mới.
    if len(np.unique(y_coords)) != len(y_coords):
        raise ValueError("LỖI: Các giá trị y (trong FileY.txt) bị trùng lặp. "
                         "Không thể dùng làm mốc nội suy cho hàm ngược.")
                         
    return x_coords, y_coords

# --- Chương trình chính cho Ô 1 ---
try:
    x_coords, y_coords = load_and_validate_inverse_data('FileX.txt', 'FileY.txt')
    print("Dữ liệu hợp lệ và đã được đọc thành công!")
    print("-" * 50)
    print("Các mốc x ban đầu:", x_coords.tolist())
    print("Các giá trị y ban đầu:", y_coords.tolist())

except (IOError, ValueError) as e:
    print(str(e))

Dữ liệu hợp lệ và đã được đọc thành công!
--------------------------------------------------
Các mốc x ban đầu: [2.265, 2.38, 2.495, 2.61, 2.725, 2.84]
Các giá trị y ban đầu: [-1.6635, -1.5819, -1.5127, -1.3977, -1.3814, -1.2792]


In [8]:
import numpy as np
import pandas as pd
from numpy.polynomial import polynomial as P

# Giả định x_coords và y_coords đã tồn tại từ các ô trước.
# Ví dụ dữ liệu (lấy từ ảnh a16a7e.png) để bạn có thể chạy thử nghiệm:
# x_coords = np.array([1.118, 1.236, 1.354, 1.472, 1.59, 1.708, 1.826])
# y_coords = np.array([1.5143, 1.9576, 2.3866, 2.7958, 3.1802, 3.5347, 3.855])

def display_lower_tri_div_diff(x_values, y_values):
    """
    Hàm này tạo, hiển thị bảng tỷ sai phân NGƯỢC dạng tam giác dưới
    và tô màu các giá trị trên đường chéo chính.
    """
    
    n = len(x_values)
    
    # --- 1. Tính toán Bảng Tỷ sai phân Ngược ---
    #    Tạo bảng ma trận (tam giác trên) để tính toán
    #    Bảng này tính f[yi, yi+1, ...] = (f[yi+1,...] - f[yi,...]) / (yi+k - yi)
    #    (Sử dụng y_values làm mốc, x_values làm giá trị)
    calc_table = np.full((n, n), np.nan)
    calc_table[:, 0] = x_values # Cột đầu tiên là x

    for j in range(1, n): # Cột (bậc tỷ sai phân)
        for i in range(n - j): # Hàng
            numerator = calc_table[i+1, j-1] - calc_table[i, j-1]
            denominator = y_values[i+j] - y_values[i]
            calc_table[i, j] = numerator / denominator
            
    # --- 2. Tạo DataFrame Hiển thị (Dạng Tam giác Dưới) ---
    #    Tạo bảng rỗng (NaN)
    display_table = np.full((n, n), np.nan)
    
    # Lặp và điền các giá trị vào đúng vị trí tam giác dưới
    for j in range(n): # Cột (bậc TSP)
        for i in range(j, n): # Hàng
            # Lấy giá trị từ bảng tính (hàng i-j, cột j)
            # Ví dụ: (hàng 2, cột 1) của bảng dưới = (hàng 1, cột 1) của bảng trên
            display_table[i, j] = calc_table[i-j, j]
            
    # Tạo DataFrame từ bảng tam giác dưới
    headers = [f'TSP{i}' for i in range(n)]
    df_display = pd.DataFrame(display_table, columns=headers)
    
    # Đổi tên cột đầu tiên thành 'x'
    df_display.rename(columns={'TSP0': 'x'}, inplace=True)
    
    # Thêm cột 'y' vào đầu
    df_display.insert(0, 'y', y_values)

    # --- 3. Hàm tô màu đường chéo ---
    def highlight_diagonal(data):
        style = 'background-color: yellow; font-weight: bold; color: black;'
        style_df = pd.DataFrame('', index=data.index, columns=data.columns)
        
        # Lặp và tô màu đường chéo
        # Bỏ qua cột 'y' (cột 0), bắt đầu từ cột 'x' (cột 1)
        for i in range(n):
            col_idx_in_df = i + 1 # Cột 'x' là 1, 'TSP1' là 2, ...
            row_idx_in_df = i
            
            if col_idx_in_df < len(data.columns):
                val = data.iloc[row_idx_in_df, col_idx_in_df]
                if not pd.isna(val): # Chỉ tô màu ô có giá trị
                    style_df.iloc[row_idx_in_df, col_idx_in_df] = style
        return style_df

    # --- 4. Áp dụng style và hiển thị ---
    print("--- Bảng Tỷ sai phân Ngược (Dạng tam giác dưới, tô màu đường chéo) ---")
    
    numeric_cols = df_display.select_dtypes(include=np.number).columns
    styled_df = df_display.style.apply(highlight_diagonal, axis=None).format(
        "{:g}", 
        subset=numeric_cols, 
        na_rep="" # Hiển thị ô trống thay vì 'NaN'
    )
    
    display(styled_df)
    
    # Trả về các hệ số Newton (Hàng đầu tiên của bảng tính toán)
    return calc_table[0, :]

# --- Thực thi hàm ---
try:
    # Chạy hàm để hiển thị bảng
    inverse_newton_coeffs = display_lower_tri_div_diff(x_coords, y_coords)
    
    # In các hệ số đã được tô màu
    print("\n--- Các hệ số Newton của đa thức P(y) (giá trị trên đường chéo, hàng đầu của bảng tính) ---")
    print(inverse_newton_coeffs)
    
except NameError:
    print("LỖI: Các biến 'x_coords' và 'y_coords' chưa được định nghĩa. Vui lòng chạy ô nạp dữ liệu trước.")

--- Bảng Tỷ sai phân Ngược (Dạng tam giác dưới, tô màu đường chéo) ---


,y,x,TSP1,TSP2,TSP3,TSP4,TSP5
0,-1.6635,2.265,,,,,
1,-1.5819,2.38,1.40931,,,,
2,-1.5127,2.495,1.66185,1.67464,,,
3,-1.3977,2.61,1,-3.5931,-19.8185,,
4,-1.3814,2.725,7.05521,46.1174,247.933,949.136,
5,-1.2792,2.84,1.12524,-50.0419,-411.817,-2179.55,-8141.26



--- Các hệ số Newton của đa thức P(y) (giá trị trên đường chéo, hàng đầu của bảng tính) ---
[ 2.26500000e+00  1.40931373e+00  1.67464181e+00 -1.98184558e+01
  9.49135579e+02 -8.14126022e+03]


In [9]:
# Giả định biến 'y_coords' đã được định nghĩa ở ô trên.

try:
    n = len(y_coords)
    
    # --- B1: Lấy các giá trị 'c' (giữ nguyên) ---
    # Lấy n-1 giá trị x đầu tiên.
    c_values = y_coords[:-1]
    
    # --- B3: Thiết lập số cột (ĐÃ THAY ĐỔI) ---
    # >>> SỬA LỖI Ở ĐÂY: Số cột bây giờ là n <<<
    num_cols = n 
    
    # Số hàng cần tính toán vẫn là n-1
    num_rows_to_calc = n - 1 

    # --- Bước khởi tạo ---
    product_table_rows = []
    
    # Hàng đầu tiên [0, 0, ..., 1] bây giờ sẽ có n phần tử
    initial_row = np.zeros(num_cols)
    initial_row[-1] = 1.0
    product_table_rows.append(initial_row)

    print("--- Bắt đầu xây dựng Bảng Tích ---")
    print(f"Số mốc nội suy n = {n}")
    print(f"Số cột trong bảng tính = {num_cols}") # Sẽ in ra 7 nếu n=7
    
    # --- B2: Lặp và tính toán các hàng (giữ nguyên) ---
    for i in range(num_rows_to_calc):
        c = c_values[i]
        a = product_table_rows[-1]
        b = np.zeros(num_cols) # Sẽ tạo hàng mới có n phần tử

        # Áp dụng công thức: b_k = a_{k+1} - a_k * c
        for k in range(num_cols):
            a_k_plus_1 = a[k+1] if k < num_cols - 1 else 0.0 
            a_k = a[k]
            b[k] = a_k_plus_1 - (a_k * c)
            
        product_table_rows.append(b)

    # --- Hiển thị kết quả ---
    product_table_data = np.array(product_table_rows)
    df_product = pd.DataFrame(product_table_data)
    
    c_column_display = [np.nan] + c_values.tolist()
    df_product.insert(0, 'Giá trị c (y)', c_column_display)

    print("\n--- Bảng Tích Hoàn Chỉnh ---")
    display(df_product.style.format("{:g}", na_rep=""))

except NameError:
    print("LỖI: Biến 'y_coords' chưa được định nghĩa. Vui lòng chạy ô nạp dữ liệu trước.")

--- Bắt đầu xây dựng Bảng Tích ---
Số mốc nội suy n = 6
Số cột trong bảng tính = 6

--- Bảng Tích Hoàn Chỉnh ---


,Giá trị c (y),0,1,2,3,4,5
0,,0,0,0,0,0,1
1,-1.6635,0,0,0,0,1,1.6635
2,-1.5819,0,0,0,1,3.2454,2.63149
3,-1.5127,0,0,1,4.7581,7.54081,3.98066
4,-1.3977,0,1,6.1558,14.1912,14.5204,5.56376
5,-1.3814,1,7.5372,22.6948,34.1242,25.6223,7.68578


In [10]:
# Giả định các biến 'inverse_newton_coeffs' và 'product_table_data' đã tồn tại.

# --- ĐỊNH NGHĨA LẠI HÀM FORMAT ĐỂ DÙNG BIẾN 'y' ---
# (Bỏ khối 'try...except' để đảm bảo hàm này được cập nhật)
print("Đang định nghĩa lại hàm 'format_polynomial_descending' để dùng biến 'y'...")
def format_polynomial_descending(coeffs_asc):
    """Hàm định dạng đa thức, sắp xếp theo bậc giảm dần."""
    coeffs_desc = coeffs_asc[::-1]; poly_str = ""; n = len(coeffs_desc) - 1
    for i, coeff in enumerate(coeffs_desc):
        if np.isclose(coeff, 0): continue
        power = n - i; sign = " - " if coeff < 0 else " + "; coeff_abs = abs(coeff)
        if poly_str == "": sign = "" if coeff > 0 else "-"
        if np.isclose(coeff_abs, 1) and power != 0: coeff_str = ""
        else: coeff_str = f"{coeff_abs:g}"
        
        # >>> ĐÃ SỬA: Thay 'x' bằng 'y' <<<
        if power == 1: var_str = "y"
        elif power == 0: var_str = ""
        else: var_str = f"y^{power}"
            
        separator = "*" if coeff_str != "" and var_str != "" else ""
        poly_str += f" {sign} {coeff_str}{separator}{var_str}"
    return poly_str.strip() if poly_str else "0"

# --- Bước 1: In Vector tham số Newton ---
print("--- B1: Vector tham số Newton (từ đường chéo Bảng Tỷ sai phân) ---")
try:
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(inverse_newton_coeffs)
except NameError:
    print("LỖI: Biến 'inverse_newton_coeffs' chưa được định nghĩa.")
    raise 
print("\n--- B1: Bảng Tích (Ma trận nhân) ---")
try:
    with np.printoptions(threshold=np.inf, precision=15, suppress=True, linewidth=np.inf):
        print(product_table_data)
except NameError:
    print("LỖI: Biến 'product_table_data' chưa được định nghĩa.")
    raise 
print("-" * 50)

# --- Bước 2: Thực hiện phép nhân ma trận ---
print("--- B2: Thực hiện phép nhân ---")
print("Vector_Newton @ Bảng_Tích")

try:
    final_coeffs_descending = inverse_newton_coeffs @ product_table_data
    
    # --- B3: In kết quả và đa thức ---
    print("\n--- Kết quả Vector hệ số cuối cùng (Hệ số P(y) bậc GIẢM DẦN) ---")
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(final_coeffs_descending)
        
    print("\n--- B3: Đa thức nội suy P(y) hoàn chỉnh (từ kết quả nhân) ---")
    
    # >>> KHÔNG CẦN THAM SỐ NỮA, VÌ HÀM ĐÃ ĐƯỢC ĐỊNH NGHĨA LẠI <<<
    print(f"P(y) =", format_polynomial_descending(final_coeffs_descending[::-1]))

except ValueError as e:
    print(f"\nLỖI KHI NHÂN MA TRẬN: {e}")
    print(f"Kích thước Vector Newton: {inverse_newton_coeffs.shape}")
    print(f"Kích thước Bảng Tích: {product_table_data.shape}")
except NameError as e:
     print(f"LỖI: Một trong các biến ('inverse_newton_coeffs' hoặc 'product_table_data') chưa được định nghĩa.")

print("=" * 50)
print("--- KẾT THÚC ---")

Đang định nghĩa lại hàm 'format_polynomial_descending' để dùng biến 'y'...
--- B1: Vector tham số Newton (từ đường chéo Bảng Tỷ sai phân) ---
[    2.265                 1.409313725490195     1.674641813610499
   -19.818455849787743   949.135579001171    -8141.260220651878   ]

--- B1: Bảng Tích (Ma trận nhân) ---
[[ 0.                 0.                 0.                 0.                 0.                 1.               ]
 [ 0.                 0.                 0.                 0.                 1.                 1.6635           ]
 [ 0.                 0.                 0.                 1.                 3.2454             2.63149065       ]
 [ 0.                 0.                 1.                 4.7581             7.54080723         3.980655906255   ]
 [ 0.                 1.                 6.155799999999999 14.191203599999998 14.520442171625998  5.563762760172613]
 [ 1.                 7.537199999999999 22.694825719999997 34.124170824665995 25.622301576056763  7.

## Thuật toán tính đạo hàm các cấp của P(y') khi y = y'

In [11]:
# Giả định biến 'final_coeffs_ascending' đã được định nghĩa ở ô trên.
# Đây là vector hệ số P(y) theo bậc tăng dần (a_0, a_1, ..., a_n).

# --- 1. Thiết lập đầu vào ---
y_prime_input = 3                                             # Bạn có thể thay đổi giá trị y' ở đây

# --- 2. Hàm thuật toán Horner lặp ---
def tinh_P_va_cac_dao_ham(coeffs_asc, y_prime):
    """
    Tính giá trị của P(y') và tất cả các đạo hàm của nó P'(y'), P''(y'),...
    bằng phương pháp Horner lặp.

    Args:
        coeffs_asc (list or np.array): Các hệ số của P(y) theo bậc TĂNG DẦN.
        y_prime (float): Điểm y' cần tính giá trị.
    """
    
    # Horner lặp hoạt động với hệ số bậc giảm dần
    coeffs_desc = coeffs_asc[::-1]
    
    print("\n" + "="*50)
    print(f"Đa thức P(y) có hệ số (bậc giảm dần): {coeffs_desc.tolist()}")
    print(f"Tính các đạo hàm tại y = {y_prime}")
    
    current_coeffs = list(coeffs_desc)
    n = len(current_coeffs) - 1
    summary_data = []
    k_factorial = 1.0

    # Lặp qua từng cấp đạo hàm, từ k = 0 đến n
    for k in range(n + 1):
        print("\n" + "="*50 + f"\n--- BƯỚC {k+1}: TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} ---")
        
        m = len(current_coeffs)
        deg = m - 1
        
        # --- Hiển thị bảng Horner cho bước hiện tại ---
        header = [f'a_{deg-i}' for i in range(deg + 1)]
        row_coeffs = [round(c, 10) for c in current_coeffs]
        row_products = ['']
        row_results = []

        b = current_coeffs[0]
        row_results.append(round(b, 10))
        
        quotient_coeffs = [b] # Hệ số của đa thức thương Q_{k+1}(x)
        
        for i in range(1, m):
            product = b * y_prime
            row_products.append(round(product, 10))
            b = product + current_coeffs[i]
            row_results.append(round(b, 10))
            if i < m - 1: # Hệ số cuối cùng là số dư
                quotient_coeffs.append(b)
        
        remainder = b # Số dư (R_{k+1})
        
        df = pd.DataFrame(
            [row_coeffs, row_products, row_results],
            index=['Hệ số vào', f'Nhân với y={y_prime}', 'Kết quả'],
            columns=header
        )
        display(df)
        
        # --- Tính giá trị đạo hàm P^(k)(y') = k! * R_{k+1} ---
        if k > 0:
            k_factorial *= k
        
        derivative_value = remainder * k_factorial
        summary_data.append([f"P^({k})(y')", derivative_value])
        
        print(f"-> Số dư R_{k+1} = {remainder:g}")
        print(f"-> Giá trị đạo hàm P^({k})(y') = R_{k+1} * {k}! = {remainder:g} * {k_factorial} = {derivative_value:g}")

        # Cập nhật hệ số cho vòng lặp tiếp theo
        current_coeffs = quotient_coeffs

    # --- In bảng tổng kết cuối cùng ---
    column_name_value = f"Giá trị tại y = {y_prime}"
    summary_df = pd.DataFrame(summary_data, columns=["Đạo hàm", column_name_value])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO y = {y_prime} ---")
    
    # Áp dụng định dạng "{:g}" CHỈ cho cột chứa số
    display(summary_df.style.format("{:g}", subset=[column_name_value]))
    
    return summary_df

# --- 3. Thực thi thuật toán ---
try:
    # Gọi hàm tính toán, sử dụng các hệ số đã tính được ở ô trước
    ket_qua_dao_ham = tinh_P_va_cac_dao_ham(final_coeffs_descending[::-1], y_prime_input)

except NameError:
    print("LỖI: Biến 'final_coeffs_ascending' chưa được định nghĩa.")
    print("Vui lòng chạy lại ô code thực hiện phép nhân ma trận trước.")


Đa thức P(y) có hệ số (bậc giảm dần): [-8141.260220651878, -60413.17095609615, -178941.61150749747, -264437.0018049385, -194958.5592536697, -57361.05932590577]
Tính các đạo hàm tại y = 3

--- BƯỚC 1: TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 ---


,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,-8141.260221,-60413.170956,-178941.611507,-2.644370e+05,-1.949586e+05,-5.736106e+04
Nhân với y=3,,-24423.780662,-254510.854854,-1.300357e+06,-4.694383e+06,-1.466803e+07
Kết quả,-8141.260221,-84836.951618,-433452.466362,-1.564794e+06,-4.889342e+06,-1.472539e+07


-> Số dư R_1 = -1.47254e+07
-> Giá trị đạo hàm P^(0)(y') = R_1 * 0! = -1.47254e+07 * 1.0 = -1.47254e+07

--- BƯỚC 2: TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 ---


,a_4,a_3,a_2,a_1,a_0
Hệ số vào,-8141.260221,-84836.951618,-433452.466362,-1.564794e+06,-4.889342e+06
Nhân với y=3,,-24423.780662,-327782.196840,-2.283704e+06,-1.154550e+07
Kết quả,-8141.260221,-109260.732280,-761234.663202,-3.848498e+06,-1.643484e+07


-> Số dư R_2 = -1.64348e+07
-> Giá trị đạo hàm P^(1)(y') = R_2 * 1! = -1.64348e+07 * 1.0 = -1.64348e+07

--- BƯỚC 3: TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 ---


...

,a_1,a_0
Hệ số vào,-8141.260221,-158108.293604
Nhân với y=3,,-24423.780662
Kết quả,-8141.260221,-182532.074266


-> Số dư R_5 = -182532
-> Giá trị đạo hàm P^(4)(y') = R_5 * 4! = -182532 * 24.0 = -4.38077e+06

--- BƯỚC 6: TÍNH TOÁN CHO ĐẠO HÀM CẤP 5 ---


,a_0
Hệ số vào,-8141.260221
Nhân với y=3,
Kết quả,-8141.260221


-> Số dư R_6 = -8141.26
-> Giá trị đạo hàm P^(5)(y') = R_6 * 5! = -8141.26 * 120.0 = -976951

--- BẢNG TỔNG HỢP KẾT QUẢ CHO y = 3 ---


,Đạo hàm,Giá trị tại y = 3
0,P^(0)(y'),-1.47254e+07
1,P^(1)(y'),-1.64348e+07
2,P^(2)(y'),-1.46707e+07
3,P^(3)(y'),-9.81968e+06
4,P^(4)(y'),-4.38077e+06
5,P^(5)(y'),-976951
